In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1=pd.read_csv('/kaggle/input/silicon-valley-diversity-data/Reveal_EEO1_for_2016.csv')

# 1.Visualization and EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df1=df1.replace('na', '0')
df1=df1.fillna({'count':'0'})
df1['count']=df1['count'].astype(int)

In [ ]:
df11=pd.DataFrame(df1[df1['job_category'] != 'Previous_totals'].groupby(['company','job_category','race','gender'])['count'].sum())
df11=df11.reset_index()
df11=df11.set_index('company')
df11=df11.pivot(columns=['job_category','race','gender'],values='count')
df11.head()

In [ ]:
df12=df11.sum(level=0,axis=1)
df12.head()

In [ ]:
df12['Totals'].sort_values(ascending=False).plot.bar(alpha=0.6, figsize=(10,10), stacked=True,title='Sort by Total Employees')

In [ ]:
df12.drop('Totals',axis=1).plot.bar(y=['Administrative support', 'Craft workers', 'Executives', 'Managers',
       'Professionals', 'Sales workers', 'Service workers', 'Technicians',
       'laborers and helpers', 'operatives'],alpha=0.6, figsize=(10,10), stacked=True, cmap='Reds',title='Job Categories by companies')

Just my idea is that

Diversity is influenced by not only how many non white and non male employees you have, but also how many non white and non male managers and executives you have. So I try to get features 'white male employees %','white male managers %'and 'white male executives %'.

I also get the features which indicate variance between 'white male employees %' and 'white male managers %', and between 'white male employees %' and 'white male executives %'.

And I also try  to find the correlation between the numbers of employees and 'white male employees %','white male managers %'and 'white male executives %'. 

How many proffesionals we have is also very important for diversity.

In [ ]:
idx = pd.IndexSlice
df11['exe_w_male%']=df11.loc[:,idx['Executives','White','male']]/df12.loc[:,'Executives']
df11['mgr_w_male%']=df11.loc[:,idx['Managers','White','male']]/df12.loc[:,'Managers']
df11['Prof%']=df12['Professionals']/df12['Totals']
df11['total_w_male%']=df11.loc[:,idx['Totals','White','male']]/df12['Totals']
df11['Grand_Total']=df12['Totals']
df11['total-exe_w_male%']=df11['exe_w_male%']-df11['total_w_male%']
df11['total-mgr_w_male%']=df11['mgr_w_male%']-df11['total_w_male%']
df11.head()

In [ ]:
df13=df11.loc[:,['Grand_Total','total-exe_w_male%','total-mgr_w_male%','total_w_male%','mgr_w_male%','exe_w_male%','Prof%']]
df13.head()

In [ ]:
sns.heatmap(df13.corr())

In [ ]:
cargs = {'kind':'bar', 'use_index':True, 'ylim': (0,None), 'rot':90}

args_list = [
    {'title': 'Total white male employees % - white male executive %', 'column':['total-exe_w_male%']},
    {'title': 'Total white male employees % - white male manager %', 'column':['total-mgr_w_male%']},
    {'title': 'Total white male %', 'column': ['total_w_male%']},
    {'title': 'white male manager %', 'column': ['mgr_w_male%']},
    {'title': 'white male executive %', 'column': ['exe_w_male%']},
    {'title': 'Professional%', 'column': ['Prof%']},
]

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(20, 20), sharex=False)

for ax, args in zip(axes.ravel(), args_list):
    df13[args['column']].plot(ax=ax,  **cargs, title=args['title'])

In [ ]:
plt.figure(figsize=(5, 5))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df13.index,y=df13['Grand_Total'],hue=df13['total_w_male%'],size=df13['Prof%'],data=df13,sizes=(1,200))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

In [ ]:
plt.figure(figsize=(5, 5))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df13.index,y=df13['Grand_Total'],hue=df13['total_w_male%'],size=df13['total-mgr_w_male%'],data=df13,sizes=(1,200))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

In [ ]:
plt.figure(figsize=(5, 5))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df13.index,y=df13['Grand_Total'],hue=df13['total_w_male%'],size=df13['total-exe_w_male%'],data=df13,sizes=(1,200))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

In [ ]:
df13.plot.bar(y=['total_w_male%','mgr_w_male%','exe_w_male%'],figsize=(20,10))

# 2. Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()
clustering_sc = sc.fit_transform(df13)

In [ ]:
pred = KMeans(n_clusters=4).fit_predict(clustering_sc)

In [ ]:
pred

In [ ]:
df13['cluster']=pred
df13['cluster'].value_counts()

In [ ]:
df13

# 3.PCA

In [ ]:
import sklearn
from sklearn.decomposition import PCA

In [ ]:
df14=df13.drop('cluster', axis=1)

In [ ]:
df14

In [ ]:
pca = PCA()
pca.fit(clustering_sc)
feature = pca.transform(clustering_sc)

In [ ]:
df15=pd.DataFrame(feature, columns=["PC{}".format(x + 1) for x in range(len(df14.columns))])
df15

In [ ]:
PC1=df15.iloc[:,0]
PC2=df15.iloc[:,1]
annotations=['23andMe', 'Adobe', 'Airbnb', 'Apple', 'Cisco', 'Facebook', 'Google',
       'HP Inc.', 'HPE', 'Intel', 'Intuit', 'LinkedIn', 'Lyft', 'MobileIron',
       'NetApp', 'Nvidia', 'PayPal', 'Pinterest', 'Salesforce', 'Sanmina',
       'Square', 'Twitter', 'Uber', 'View', 'eBay']
plt.figure(figsize=(6, 6))
plt.scatter(feature[:, 0], feature[:, 1], alpha=0.8, c=list(df14.iloc[:, 0]))
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
for i, label in enumerate(annotations):
    plt.annotate(label, (PC1[i], PC2[i]))

plt.show()

In [ ]:
pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(len(df14.columns))])

In [ ]:
import matplotlib.ticker as ticker
plt.gca().get_xaxis().set_major_locator(ticker.MaxNLocator(integer=True))
plt.plot([0] + list( np.cumsum(pca.explained_variance_ratio_)), "-o")
plt.xlabel("Number of principal components")
plt.ylabel("Cumulative contribution rate")
plt.grid()
plt.show()

In [ ]:
pd.DataFrame(pca.explained_variance_, index=["PC{}".format(x + 1) for x in range(len(df14.columns))])

In [ ]:
pd.DataFrame(pca.components_, columns=df13.columns[1:], index=["PC{}".format(x + 1) for x in range(len(df14.columns))])

In [ ]:
plt.figure(figsize=(6, 6))
for x, y, name in zip(pca.components_[0], pca.components_[1], df13.columns[1:]):
    plt.text(x, y, name)
plt.scatter(pca.components_[0], pca.components_[1], alpha=0.8)
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()